2010 Census Resources

http://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?src=CF

https://www.opendataphilly.org/dataset/census-tracts


In [1]:
import json

import pandas as pd

In [2]:
census_data = pd.read_csv('census_data/DEC_10_SF1_GCTPH1.CY07_with_ann.csv', skiprows=[0])
census_data.head()

,Id,Id2,Geography,Target Geo Id,Target Geo Id2,Geographic area,Geographic area.1,Population,Housing units,Area in square miles - Total area,Area in square miles - Water area,Area in square miles - Land area,Density per square mile of land area - Population,Density per square mile of land area - Housing units
0,0500000US42101,42101,"Philadelphia County, Pennsylvania",0500000US42101,42101,Philadelphia County,Philadelphia County,1526006,670171,142.71,8.61,134.10,11379.5,4997.5
1,0500000US42101,42101,"Philadelphia County, Pennsylvania",1400000US42101000100,42101000100,Philadelphia County - Census Tract 1,Census Tract 1,3478,2696,0.27,0.00,0.27,12778.9,9905.7
2,0500000US42101,42101,"Philadelphia County, Pennsylvania",1400000US42101000200,42101000200,Philadelphia County - Census Tract 2,Census Tract 2,2937,1285,0.15,0.00,0.15,19888.2,8701.5
3,0500000US42101,42101,"Philadelphia County, Pennsylvania",1400000US42101000300,42101000300,Philadelphia County - Census Tract 3,Census Tract 3,3169,1968,0.21,0.00,0.21,14968.2,9295.5
4,0500000US42101,42101,"Philadelphia County, Pennsylvania",1400000US42101000401,42101000401,Philadelphia County - Census Tract 4.01,Census Tract 4.01,2125,1790,0.09,0.01,0.08,25626.6,21586.7


In [3]:
# Skip first row of total population
total_population = census_data.Population.iloc[0]
print total_population
census_data = census_data[['Target Geo Id2', 'Population']].iloc[1:]
census_data.head()

1526006


,Target Geo Id2,Population
1,42101000100,3478
2,42101000200,2937
3,42101000300,3169
4,42101000401,2125
5,42101000402,3142


In [4]:
# Sort by population descending and calculate cumulative sum
sorted_population = census_data.sort('Population', ascending=False)
sorted_population['Cumulative Population'] = sorted_population.Population.cumsum()
sorted_population.head()

,Target Geo Id2,Population,Cumulative Population
182,42101018800,8322,8322
373,42101039000,8154,16476
185,42101019200,7972,24448
184,42101019100,7937,32385
52,42101004102,7848,40233


In [5]:
# Grab census geo ids where the cumulative population is less than half city population
below_50 = (
    sorted_population[sorted_population['Cumulative Population'] < (total_population /2.)]
    )["Target Geo Id2"].values
below_50

array([42101018800, 42101039000, 42101019200, 42101019100, 42101004102,
       42101989100, 42101034400, 42101033000, 42101019000, 42101030000,
       42101006700, 42101030200, 42101035500, 42101028600, 42101025200,
       42101030600, 42101017500, 42101016000, 42101019700, 42101008802,
       42101036900, 42101031300, 42101032600, 42101033600, 42101027402,
       42101008200, 42101031000, 42101036303, 42101003600, 42101017800,
       42101026600, 42101031401, 42101003701, 42101032000, 42101026700,
       42101006000, 42101034701, 42101028902, 42101017900, 42101008500,
       42101004101, 42101031501, 42101019800, 42101028300, 42101003300,
       42101029000, 42101036202, 42101015200, 42101011400, 42101031700,
       42101027300, 42101003901, 42101031600, 42101033800, 42101035800,
       42101030100, 42101020700, 42101030502, 42101021700, 42101032500,
       42101015800, 42101005500, 42101037700, 42101011800, 42101031402,
       42101016100, 42101034100, 42101009000, 42101017601, 42101

In [6]:
# Load census data from OpenData
with open("/Users/pjob/Downloads/Census_Tracts_2010.geojson") as gj:
    geojson = json.load(gj)

In [7]:
# Grab feature from geojson and match census tract geo id from census data
features = geojson['features']
for f_num, feature in enumerate(geojson['features']):
    if int(feature['properties']['GEOID10']) in below_50:
        feature['properties']['fill'] = "#ef2704"
        feature['properties']['fill-opacity'] = 0.8
    geojson['features'][f_num] = feature

In [8]:
# Save geojson
with open('census_data/philly50.geojson', 'w') as ojson:
    json.dump(geojson, ojson)